# Install gdown Python package.

In [0]:
!pip install -U --no-cache-dir gdown

# Install Tensorflow-Addons.

In [0]:
!pip install tensorflow-addons

# Install Tensorflow-datasets.

In [0]:
!pip install tensorflow-datasets

# Use TensorFlow 2.x.

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow.keras.layers as layers

print(tf.__version__)

# Create the optimizer.

*   Adam optimizer
*   Learning rate = 0.0002
*   β1 = 0.5
*   β2 = 0.999

In [0]:
optimizer = tf.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)

# List available TensorFlow datasets.

In [0]:
import tensorflow_datasets as tfds

tfds.list_builders()

# Create AttGAN encoder model.

In [0]:
class UNetGenc(layers.Layer):

  def __init__(self, dimension=64, downsamplings_layers=5):
    super(UNetGenc, self).__init__()

    self._dimension = 64
    self._downsamplings_layers = 5

  def call(self, inputs):

    input_layer = inputs
    output_units = self._dimension
    
    output_layers = []
    for layer_index in range(self._downsamplings_layers):
      input_layer = layers.Conv2D(output_units, (4,4), strides=(2,2), padding='same')(input_layer)
      input_layer = layers.BatchNormalization()(input_layer)
      input_layer = layers.LeakyReLU(alpha=0.2)(input_layer)

      output_layers.append(input_layer)
      output_units = output_units * 2    
    
    return(output_layers)

# Create AttGAN decoder model.

In [0]:
def concatenate(a_list, b_list=[]):
  # tile all elements of `b_list` and then concat `a_list + b_list` along the channel axis
  # `a` shape: (N, H, W, C_a)
  # `b` shape: can be (N, 1, 1, C_b) or (N, C_b)
  a_list = list(a_list) if isinstance(a_list, (list, tuple)) else [a_list]
  b_list = list(b_list) if isinstance(b_list, (list, tuple)) else [b_list]
  for i, b in enumerate(b_list):
        b = tf.reshape(b, [-1, 1, 1, b.shape[-1]])
        b = tf.tile(b, [1, a_list[0].shape[1], a_list[0].shape[2], 1])
        b_list[i] = b
  return tf.concat(a_list + b_list, axis=-1)

class UNetGdec(layers.Layer):

  def __init__(self, dimension=64, upsamplings_layers=5, shortcut_layers=1, inject_layers=1):
    super(UNetGdec, self).__init__()

    self._dimension = 64
    self._upsamplings_layers = 5
    self._shortcut_layers = shortcut_layers
    self._inject_layers = inject_layers

  def call(self, inputs):
    zs, a = inputs

    a = tf.to_float(a)
    input_layer = concatenate(zs[-1], a)
    output_units = self._dimension
    for layer_index in range(self.__upsamplings_layers-1):
      input_layer = layers.Conv2DTranspose(output_units, (4, 4), strides=(2,2), padding='same')(input_layer)
      input_layer = layers.BatchNormalization()(input_layer)
      input_layer = layers.LeakyReLU(alpha=0.2)(input_layer)

      if (self._shortcut_layers > layer_index):
        input_layer = concatenate([input_layer, zs[-2 - layer_index]])

      if (self._inject_layers > layer_index):
        input_layer = concatenate(input_layer, a)

      output_units = output_units * 2

    input_layer = layers.Conv2DTranspose(3, (4, 4), strides=(2,2), padding='same')(input_layer)
    input_layer = tf.keras.activations.tanh(input_layer) 

    output_layer = input_layer

    return(output_layer)  

# Create AttGAN discriminator model.

In [0]:
import tensorflow_addons as tfa

class Discriminator(layers.Layer):

  def __init__(self, number_of_attributes, dimension=64, dense_dimension=1024, downsamplings_layers=5): 
    super(Discriminator, self).__init__()  

    self._number_of_attributes = number_of_attributes
    self._dimension = dimension
    self._dense_dimension = dense_dimension
    self._downsamplings_layers = downsamplings_layers

    def call(self, inputs):

      input_layer = inputs  
      output_units = self._dimension  

      for layer_index in range(self._downsamplings_layers): 
        input_layer = layers.Conv2D(output_units, (4,4), strides=(2,2), padding='same')(input_layer)         
        input_layer = tfa.layers.InstanceNormalization()(input_layer)
        input_layer = layers.LeakyReLU(alpha=0.2)(input_layer)

        output_units = output_units * 2

      input_layer = layers.Flatten()(input_layer)

      discriminator_output = layers.Dense(self._dense_dimension)(input_layer)      
      discriminator_output = layers.LeakyReLU(alpha=0.2)(discriminator_output)
      discriminator_output = layers.Dense(1)(discriminator_output)
      
      attribute_output = layers.Dense(self._dense_dimension)(input_layer)      
      attribute_output = layers.LeakyReLU(alpha=0.2)(attribute_output)
      attribute_output = layers.Dense(self._number_of_attributes)(attribute_output)

      return(discriminator_output, attribute_output)